In [12]:
import pandas as pd

df = pd.read_csv("date.csv").drop_duplicates(subset = "informal_date")
df.shape

(9251, 2)

#Normalize and split Data

In [21]:
import re
from sklearn.model_selection import train_test_split

def normalize_text(text):
    # Convert Persian numbers to Latin
    persian_to_latin = {
        '۰': '0', '۱': '1', '۲': '2', '۳': '3', '۴': '4',
        '۵': '5', '۶': '6', '۷': '7', '۸': '8', '۹': '9'
    }
    for persian, latin in persian_to_latin.items():
        text = text.replace(persian, latin)
    
    # Normalize Persian characters
    text = text.replace('ي', 'ی').replace('ك', 'ک')
    
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text.strip())
    text = f"<start> {text} <ends>"
    return text

df['formal_date'] = df['formal_date'].apply(normalize_text)
df['informal_date'] = df['informal_date'].apply(normalize_text)

df

,informal_date,formal_date
0,<start> پنجم اردیبهشت 1402 <ends>,<start> 1402/02/05 <ends>
1,<start> 17 آبان 1395 <ends>,<start> 1395/08/17 <ends>
2,<start> 1 1403 مهر <ends>,<start> 1403/07/01 <ends>
3,<start> سال مهر اول 1403 <ends>,<start> 1403/07/01 <ends>
4,<start> 1403-07-01 <ends>,<start> 1403/07/01 <ends>
...,...,...
29435,<start> پانزدهم آبان 1307 <ends>,<start> 1307/08/15 <ends>
29436,<start> هجدهم آذر 1306 <ends>,<start> 1306/09/18 <ends>
29437,<start> بیست و دوم دی 1305 <ends>,<start> 1305/10/22 <ends>
29439,<start> پنجم اسفند 1303 <ends>,<start> 1303/12/05 <ends>


In [32]:
train_val , test = train_test_split(df , test_size=0.1 , random_state= 42)
train ,val = train_test_split(train_val,test_size=0.1,random_state=42)

In [33]:
print(f"Training samples: {len(train)}")
print(f"Validation samples: {len(val)}")
print(f"Test samples: {len(test)}")

Training samples: 7492
Validation samples: 833
Test samples: 926


In [34]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, device):
    best_val_loss = float('inf')
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for batch in train_loader:
            input_seq, target_seq = batch
            input_seq, target_seq = input_seq.to(device), target_seq.to(device)
            
            optimizer.zero_grad()
            output = model(input_seq)
            loss = criterion(output.view(-1, output.size(-1)), target_seq.view(-1))
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
        
        avg_train_loss = total_loss / len(train_loader)
        
        # Validation
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for batch in val_loader:
                input_seq, target_seq = batch
                input_seq, target_seq = input_seq.to(device), target_seq.to(device)
                output = model(input_seq)
                loss = criterion(output.view(-1, output.size(-1)), target_seq.view(-1))
                val_loss += loss.item()
        
        avg_val_loss = val_loss / len(val_loader)
        
        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")
        
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), 'best_model.pth')
            print("Model saved!")

# Assuming you have your data prepared and model defined
model = YourTransformerModel()  # Define your model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss(ignore_index=pad_token_id)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Prepare your data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

num_epochs = 10
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, device)

ModuleNotFoundError: No module named 'torch'